# Project: [Analyzing Students' Mental Health in SQL](https://app.datacamp.com/learn/projects/analyzing_students_mental_health)
![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using DuckDB to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

### Setup our environment and load in our data

JupySQL reference information:

- [Options](https://jupysql.ploomber.io/en/latest/api/configuration.html#options) - for the config SqlMagic settings

In [1]:
%load_ext sql
%sql duckdb:// --alias students
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False

Connecting to 'students'

In [2]:
%%sql
CREATE TABLE students AS FROM 'data/students.csv'

,Success


We can use the JupySQL [%sqlcmd profile](https://jupysql.ploomber.io/en/latest/api/magic-profile.html#sqlcmd-profile) to obtain summary statistics of the table. **Note:** the count used in the profile command excludes NULLS.

In [3]:
%sqlcmd profile --table students

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
count,268,268,268,268,268,268,268,268,268,268,268,268,260,268,268,270,271,268,273,268,268,268,268,268,268,268,268,268,268,268,268,268,268,268,268,268,268,268,242,283,283,272,272,272,272,272,272,272,272,272
unique,2,5,2,2,14,5,9,3,5,3,5,3,2,2,2,4,6,26,10,38,27,15,18,14,11,9,32,80,7,7,7,7,7,7,7,7,7,7,7,14,14,4,4,4,4,4,4,4,4,4
top,Inter,SEA,Female,Under,nan,nan,nan,Medium,nan,Low,nan,High,No,No,No,No,No,nan,Mild,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1,Yes,No,Yes,No,No,No,No,No,No,No,No
freq,201,122,170,247,nan,nan,nan,121,nan,92,nan,166,157,177,207,172,172,nan,107,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,68,145,140,137,202,207,238,222,249,203,247,223
mean,nan,nan,nan,nan,20.8731,2.5597,2.1493,nan,3.0970,nan,3.6493,nan,nan,nan,nan,nan,nan,8.1866,nan,37.4739,15.4142,9.6082,9.1381,7.2575,6.0634,3.7836,21.1157,72.3806,4.3209,4.0634,4.3731,3.0709,2.9478,2.2873,2.6716,1.9179,2.9403,2.1418,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
std,nan,nan,nan,nan,2.7601,1.0686,1.3243,nan,1.3037,nan,0.8789,nan,nan,nan,nan,nan,nan,4.9455,nan,9.2169,6.1629,4.0037,4.1868,3.1059,2.5959,1.9059,7.3883,22.6026,2.2297,1.9316,2.0685,1.8504,1.8257,1.5270,1.7116,1.4039,2.0269,1.5020,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
min,nan,nan,nan,nan,17,1,1,nan,1,nan,1,nan,nan,nan,nan,nan,nan,0,nan,8,8,4,5,4,3,2,10,36,1,1,1,1,1,1,1,1,1,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
25%,nan,nan,nan,nan,19.0000,2.0000,1.0000,nan,2.0000,nan,3.0000,nan,nan,nan,nan,nan,nan,5.0000,nan,31.0000,10.0000,6.0000,5.0000,4.0000,3.0000,2.0000,15.0000,56.0000,2.0000,2.0000,3.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
50%,nan,nan,nan,nan,20.0000,2.0000,2.0000,nan,3.0000,nan,4.0000,nan,nan,nan,nan,nan,nan,8.0000,nan,40.0000,16.0000,9.0000,9.0000,7.0000,6.0000,4.0000,20.0000,72.0000,5.0000,4.0000,5.0000,3.0000,3.0000,2.0000,2.0000,1.0000,2.0000,1.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
75%,nan,nan,nan,nan,22.0000,3.0000,3.0000,nan,4.0000,nan,4.0000,nan,nan,nan,nan,nan,nan,11.0000,nan,46.0000,19.0000,12.0000,11.0000,9.0000,8.0000,5.0000,26.0000,88.0000,6.0000,5.0000,6.0000,4.0000,4.0000,3.0000,4.0000,3.0000,4.0000,3.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


DuckDB has some similiar functionality with the [SUMMARIZE](https://duckdb.org/docs/guides/meta/summarize) command which also includes statistics on the number of missing records (nulls). 


In [4]:
%sql SUMMARIZE students;

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,inter_dom,VARCHAR,Dom,Inter,2,None,None,None,None,None,286,6.29%
1,region,VARCHAR,EA,SEA,5,None,None,None,None,None,286,6.29%
2,gender,VARCHAR,Female,Male,2,None,None,None,None,None,286,6.29%
3,academic,VARCHAR,Grad,Under,2,None,None,None,None,None,286,6.29%
4,age,BIGINT,17,31,14,20.87313432835821,2.7652786783892576,19,20,22,286,6.29%
5,age_cate,BIGINT,1,5,5,2.5597014925373136,1.0706237044601954,2,2,3,286,6.29%
6,stay,BIGINT,1,10,9,2.1492537313432836,1.326804969095303,1,2,3,286,6.29%
7,stay_cate,VARCHAR,Long,Short,3,None,None,None,None,None,286,6.29%
8,japanese,BIGINT,1,5,5,3.0970149253731343,1.306104382870079,2,3,4,286,6.29%
9,japanese_cate,VARCHAR,Average,Low,3,None,None,None,None,None,286,6.29%


Based on the output we see that we have:
- 286 rows and 50 columns of data
- between 1 and 15 % of columns have some missing data
- looking at the original csv file we see that the last 18 lines of the file have mainly missing data which suggests the file might have been altered or corrupted.

### Start by counting all of the records in the data, then all records per student type to see how the records are categorized and scored.

We can use [FROM first](https://duckdb.org/2023/08/23/even-friendlier-sql.html#from-first-in-select-statements) with DuckDB because that's the way it should have always been! ;)



In [5]:
%%sql
-- count the number of record for each type of student
FROM students
SELECT 
	COUNT(*) AS total_records,
	COUNT(inter_dom) AS inter_dom_count,
	COUNT(*) FILTER (WHERE inter_dom == 'Inter') AS inter_count,
	COUNT(*) FILTER (WHERE inter_dom == 'Dom') AS dom_count,
	COUNT(*) FILTER (WHERE inter_dom IS NULL) AS missing_inter_dom_count;

,total_records,inter_dom_count,inter_count,dom_count,missing_inter_dom_count
0,286,268,201,67,18


Another way is to use [histogram](https://duckdb.org/docs/sql/aggregates#general-aggregate-functions) to get a key-value pairing of our data.

In [6]:
%%sql
FROM students
SELECT histogram(inter_dom);

,histogram(inter_dom)
0,"{'key': ['Dom', 'Inter'], 'value': [67, 201]}"


### Filter the data to see how it differs between the student types.

In [7]:
%%sql 
-- Review the data of international students
SUMMARIZE FROM students WHERE inter_dom = 'Inter';

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,inter_dom,VARCHAR,Inter,Inter,1,None,None,None,None,None,201,0.0%
1,region,VARCHAR,EA,SEA,5,None,None,None,None,None,201,0.0%
2,gender,VARCHAR,Female,Male,2,None,None,None,None,None,201,0.0%
3,academic,VARCHAR,Grad,Under,2,None,None,None,None,None,201,0.0%
4,age,BIGINT,17,31,14,21.029850746268657,3.033002551534032,19,20,22,201,0.0%
5,age_cate,BIGINT,1,5,5,2.601990049751244,1.1272958883542945,2,2,3,201,0.0%
6,stay,BIGINT,1,10,9,2.0646766169154227,1.375062187648434,1,2,3,201,0.0%
7,stay_cate,VARCHAR,Long,Short,3,None,None,None,None,None,201,0.0%
8,japanese,BIGINT,1,5,5,2.5223880597014925,0.9114528340274755,2,3,3,201,0.0%
9,japanese_cate,VARCHAR,Average,Low,3,None,None,None,None,None,201,0.0%


In [8]:
%%sql
-- Review the data of domestic students
SUMMARIZE FROM students WHERE inter_dom = 'Dom';

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,inter_dom,VARCHAR,Dom,Dom,1,None,None,None,None,None,67,0.0%
1,region,VARCHAR,JAP,JAP,1,None,None,None,None,None,67,0.0%
2,gender,VARCHAR,Female,Male,2,None,None,None,None,None,67,0.0%
3,academic,VARCHAR,Grad,Under,2,None,None,None,None,None,67,0.0%
4,age,BIGINT,18,27,8,20.402985074626866,1.661139637030328,19,20,21,67,0.0%
5,age_cate,BIGINT,1,5,5,2.4328358208955225,0.8742768543724359,2,2,3,67,0.0%
6,stay,BIGINT,1,5,5,2.4029850746268657,1.1422956881507507,1,3,3,67,0.0%
7,stay_cate,VARCHAR,Long,Short,3,None,None,None,None,None,67,0.0%
8,japanese,BIGINT,2,5,4,4.82089552238806,0.601144694282912,5,5,5,67,0.0%
9,japanese_cate,VARCHAR,Average,Low,3,None,None,None,None,None,67,0.0%


### Find the summary statistics of the diagnostic tests for all students using aggregate functions, rounding the test scores to two decimal places, remembering to use aliases.
The diagnotistic tests are under columns:
- `todep`
- `tosc`
- `toas`

In [9]:
%%sql
-- Domestic student test scores
FROM students
SELECT 
	stay, 
	ROUND(AVG(todep), 2) AS average_phq, -- PHQ test
	ROUND(AVG(tosc), 2) AS average_scs, -- SCS test
	ROUND(AVG(toas), 2) AS average_as -- ASISS test
WHERE inter_dom = 'Dom'
GROUP BY stay
ORDER BY stay DESC;	

,stay,average_phq,average_scs,average_as
0,5,11.50,34.00,88.00
1,4,7.00,36.67,64.78
2,3,8.43,39.09,58.04
3,2,9.46,37.08,66.46
4,1,8.70,37.15,62.60


Your final query:

- See if length of stay impacts the average diagnostic scores rounded to two decimal places for international students, and order the results by descending order of the length of stay.
- Return a table with nine observations and five columns: `stay`, `count_int`, `average_phq`, `average_scs`, and `average_as`, **in that order**. The average columns should contain the average of the `todep`, `tosc`, and `toas` columns for each length of stay, **rounded to two decimal places**. The `count_int` column should be the number of international students for each length of stay. Sort the results by the length of stay in **descending order**.

In [10]:
%%sql
-- International student test scores
FROM students
SELECT 
	stay, 
	COUNT(*) AS count_int,
	ROUND(AVG(todep), 2) AS average_phq, -- PHQ test
	ROUND(AVG(tosc), 2) AS average_scs, -- SCS test
	ROUND(AVG(toas), 2) AS average_as -- ASISS test
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;	

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


It seems that our data is potentially missing records from international students that have stayed 5 years for their PHQ test scores. 

Assuming that a higher score is meant to show a higher level of depression it would appear there might be some level of correlation between a higher length of stay and a higher risk of depression. 

In [30]:
%%sql
SELECT 
    CORR(todep, stay) AS phq_stay_corr,
    CORR(tosc, stay) AS scs_stay_corr,
    CORR(toas, stay) AS asiss_stay_corr
FROM students;

,phq_stay_corr,scs_stay_corr,asiss_stay_corr
0,0.051581,-0.037587,0.000221
